# Funções e rotinas base para o HSPmol
O objetivo desse conjunto de algoritmos é determinar os parâmetros de solubilidade de Hansen através do método de soma de contribuições de grupos proposto por Mathieu e através do método de otimização da função de desejabilidade utilizando dados de ensaios experimentais qualitativos.

In [37]:
# Instala as dependências
!pip install bioinfokit
!pip install pint
!pip install pint-pandas
!pip install SciencePlots
! pip install rdkit-pypi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 36.8 MB 33 kB/s 


In [38]:
# Carrega as bibliotecas:
import sys
import pandas as pd
import pint #(controle de unidades!)
import pint_pandas
import numpy as np
from math import sqrt
from math import log
from math import e
import math
import scipy.stats as stats
from scipy.stats import gmean
from scipy.stats import normaltest
from scipy.optimize import minimize
import statsmodels.api as sm # Para o anova
from statsmodels.formula.api import ols # Para o anova
from bioinfokit.analys import stat # Outra versão de anova

from collections import Counter, OrderedDict, defaultdict

import pandas.util.testing as tm
from pandas.core.arrays import categorical

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdDepictor
from rdkit.Chem import Descriptors
rdDepictor.SetPreferCoordGen(True)
from rdkit.Chem import Draw
from rdkit.Chem.Draw import rdMolDraw2D
from IPython.display import SVG
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import PandasTools

from sklearn.metrics import r2_score

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
#pd.options.plotting.backend = "plotly"

import matplotlib.pyplot as plt
plt.style.reload_library()
plt.style.use(['science', 'notebook'])
%matplotlib inline

#u = pint.UnitRegistry()
#3 * u.meter + 4 * u.cm

In [69]:
# Carrega a base de dados dos solventes padrão obtidos no site do professor Abbott em janeiro de 2022.
solventesAbbott = 'https://github.com/edtrawtmam/hsp/blob/7294425a3d7cedd51d295773183cc9b8acc70049/SolventesAbbott.tsv?raw=True'

# Cria os datasets com os dados digitalizados
biblio_solventesAbbott = pd.read_csv(solventesAbbott, sep='\t', header=0)

# Remove informação desnecessária da base de dados do Abbott, trazida de outros experimentos...
biblio_solventesAbbott.drop(['semaforo_NEV', 'semaforo_4BZC', 'semaforo_BZC', 'semaforo_TEO', 'semaforo_AS', 'semaforo_CAF', 'semaforo_ASC', 'semaforo_ADPC', 'semaforo_AAS'], axis=1, inplace=True)

# Legenda deste semáforo, obtido no Handbook do Hansen, pág. 507, Apendix A, Table A3.
# 1. Soluble
# 2. Almost soluble
# 3. Strongly swollen, slight solubility
# 4. Swollen
# 5. Little swelling
# 6. No visible effect
#

# Relata o tamanho do dataframe de solventes final
print('Total de solventes na lista de Abbott: ', len(biblio_solventesAbbott))


Total de solventes na lista de Abbott:  1218


In [70]:
biblio_solventesAbbott # Exibe o dataframe

,Smile,Nome,CID,CAS,Apelido,dd,dp,dh,classe,Metodo
0,C(C(Cl)(Cl)Cl)Cl,"1,1,1,2-Tetrachloroethane",12418,630-20-6,NaN,18.0,4.4,4.2,solv,Abbott
1,CC(Cl)(Cl)Cl,"1,1,1-Trichloroethane",6278,71-55-6,NaN,16.8,4.3,2.0,solv,Abbott
2,CC(F)(F)F,"1,1,1-Trifluoroethane",9868,420-46-2,NaN,14.6,10.0,0.0,solv,Abbott
3,C(C(Br)Br)(Br)Br,"1,1,2,2-Tetrabromoethane",6588,79-27-6,NaN,21.0,7.0,8.2,solv,Abbott
4,C(C(Cl)Cl)(Cl)Cl,"1,1,2,2-Tetrachloroethane",6591,79-34-5,NaN,18.8,5.1,5.3,solv,Abbott
...,...,...,...,...,...,...,...,...,...,...
1213,C1COC1=O,b-Propiolactone,2365,57-57-8,NaN,17.0,18.2,9.0,solv,Abbott
1214,C1=CC=C(C=C1)C(=O)CCl,a-Chloro Acetophenone,10757,532-27-4,NaN,20.0,9.6,5.7,solv,Abbott
1215,COC(=O)C(=C)Cl,a-Chloro Methyl Acrylate,6659,80-63-7,NaN,15.9,7.3,8.5,solv,Abbott
1216,C1CC(=O)OC1,g-Butyrolactone (GBL),7302,96-48-0,NaN,18.0,16.6,7.4,solv,Abbott


In [158]:
# Sequencia de funções

# Algoritmo modificado de Mathieu para determinação do HSP à partir da entrada do SMILE da substância:
# Funciona apenas para moléculas que contenha carbono.
# FOR MOLAR VOLUME AND REFRACTIVITY
# additive increments
params = (('B30', 7.61, 2.91),
          ('Br10', 27.52, 8.44),
          ('C10', 14.68, 2.32),
          ('C20', 9.89, 4.01),
          ('C21', 22.77, 4.58),
          ('C30', -0.00, 3.15),
          ('C30a', -0.00, 3.48),
          ('C31', 13.23, 4.65),
          ('C31a', 13.23, 4.46),
          ('C32', 27.17, 5.38),
          ('C40', -8.40, 2.60),
          ('C41', 4.46, 3.48),
          ('C42', 16.57, 4.60),
          ('C43', 29.58, 5.74),
          ('Cl10', 24.74, 5.87),
          ('F10', 17.63, 1.06),
          ('Ge40', 9.36, 8.54),
          ('I10', 35.64, 13.75),
          ('N10', 14.09, 1.55),
          ('N20', 7.42, 2.60),
          ('N20a', 7.42, 2.44),
          ('N21', 18.14, 3.55),
          ('N30', -3.08, 2.89),
          ('N30a', -3.08, 2.85),
          ('N31', 7.74, 3.69),
          ('N31a', 7.74, 3.72),
          ('N32', 17.81, 4.60),
          ('N43', 5.36, 8.02),
          ('O10', 14.89, 1.84),
          ('O20', 6.25, 1.55),
          ('O20a', 6.25, 0.71),
          ('O21', 11.78, 2.51),
          ('P30', 10.42, 7.41),
          ('P40', -1.94, 4.98),
          ('P41', 10.06, 5.41),
          ('R5', 9.41, None),
          ('R6', 6.89, None),
          ('R<5', 10.89, None),
          ('R>6', 3.75, None),
          ('S10', 25.92, 10.60),
          ('S20', 14.90, 8.22),
          ('S20a', 14.90, 7.05),
          ('S21', 26.14, 8.71),
          ('S30', 5.58, 7.28),
          ('S40', -3.74, 5.12),
          ('Se20', 19.00, 11.53),
          ('Se20a', 19.00, 9.43),
          ('Si40', 9.28, 6.38),
          ('Si41', 23.35, 7.85),
          ('Sn40', 14.47, 14.90),
          ('Ti40', 6.09, 14.66),
          ('aromat', 1.82, None))
vi = dict((p[0], p[1]) for p in params)  # volume increments
ri = dict((p[0], p[2]) for p in params)  # molar refractivity increments

# FOR POLAR (P) HSP COMPONENT
params_p = {
    'N(1)': 2783.0,
    'N(2)': 8235.0,
    'O(0)': 1603.0,
    'O(1)': 4125.0,
    'Cl(0)': 1637.0,
    'C=O': 7492.0,
    'COOH': -5494.0,
    'COinAmide': 15972.0,
    'Carbonate': 19019.0,
    'Ester': 3653.0,
    'C#N': 16056.0,
    'NitroOnC': 13276.0,
    'O=P': 20310.0,
}

POLARGROUPS = OrderedDict([
  ("COOH", ("[CX3](=O)[OX2H1]"                          , (1,2))),  # =O in carboxilic acid
  ("NitroOnC", ("[#6][$([NX3](=O)=O),$([NX3+](=O)[O-])]", (1,))),  # N in nitroaliphatic
  ("Carbonate", ("[OX2;R][CX3;R](=O)[OX2]"              , (0,1,3))),  # Carbonate
  ("Ester", ("[OX2][CX3]=O"                             , (1,))),  # C in cyclic ester
  ("COinAmide", ("[NX3][CX3](=[OX1])[#6]"               , (1,))),
  ("SO2", ("O=S=O"                                      , (1,))),  # S in sulfone
])

# FOR HYDROGEN-BONDING (H) HSP COMPONENT
params_h = {
    'HC': 24.5,
    'HN': -1576.0,
    'HNamide': 5060.0,
    'H2N': 5484.0,
    'HO': 16945.0,
    'HO_COOH': 7094.0,
    'N': 3252.0,
    'O': 1980.0,
    'X': 412.0,
}


#@title
def get_ring_descriptors(mol, maxi=6, mini=5):
    """ return dict of ring descriptors for molecule provided as input """
    dic = Counter()
    ri = mol.GetRingInfo()
    for ring in ri.AtomRings():
        size = len(ring)
        if size > maxi:
            label = 'R>'+str(maxi)
        elif size < mini:
            label = 'R<'+str(mini)
        else:
            label = 'R%u' %len(ring)
        dic[label] += 1
        # contribute also +1 aromatic ring ?
        atoms = [mol.GetAtomWithIdx(i) for i in ring]
        if all(at.GetIsAromatic() for at in atoms):
            dic['aromat'] += 1
    return dic

def GetBChar(bond):
    if bond.GetBondType()==Chem.rdchem.BondType.AROMATIC: return '~'
    if bond.GetBondType()==Chem.rdchem.BondType.DOUBLE: return '='
    if bond.GetBondType()==Chem.rdchem.BondType.TRIPLE: return '#'
    return '-'

def get_nH(atom):
    nD = len([x for x in atom.GetNeighbors() if x.GetMass()>2 and x.GetSymbol()=='H'])
    nH = nD + atom.GetTotalNumHs()
    return nH

def isNitroN(at, mol):
    if at.GetSymbol() != "N": return False
    if at.GetTotalDegree() != 3: return False
    voisins = at.GetNeighbors()
    Os = [a for a in voisins if a.GetSymbol()=="O"]
    O1s = [a for a in Os if a.GetTotalDegree()==1]
    return len(O1s) > 1
    
def isAmideN(at, mol):
    amideSMARTS = "[NX3][CX3](=[OX1])[#6]"
    amidePattern = Chem.MolFromSmarts(amideSMARTS)
    N_indices = [t[0] for t in mol.GetSubstructMatches(amidePattern)]
    return at.GetIdx() in N_indices
    
def inCOOH(at, mol):
    acSMARTS = "[CX3](=O)[OX2H1]"
    acPattern = Chem.MolFromSmarts(acSMARTS)
    OH_indices = [t[2] for t in mol.GetSubstructMatches(acPattern)]
    return at.GetIdx() in OH_indices

def get_polar_groups(mol):
    global POLARGROUPS
    by_type = defaultdict(list)
    counted_indices = set()
    # first count complex polar groups
    for group_name in POLARGROUPS:
        group, positions = POLARGROUPS[group_name]
        pattern = Chem.MolFromSmarts(group)
        tuples = mol.GetSubstructMatches(pattern)
        for tup in tuples:
            if set(tup) & counted_indices: continue
            counted_indices |= set(tup)
            by_type[group_name].append(1)
    # count insaturated polar bonds
    for bond in mol.GetBonds():
        order = GetBChar(bond)
        if order in ("#", "=", "~"):
            abeg, aend = bond.GetBeginAtom(), bond.GetEndAtom()
            tup = (abeg.GetIdx(), aend.GetIdx())
            if set(tup) & counted_indices: continue
            symbols = sorted([abeg.GetSymbol(), aend.GetSymbol()])
            if set(symbols) == set(["C"]): continue
            bondsymbol = order.join(symbols)
            counted_indices |= set(tup)
            by_type[bondsymbol].append(order)
    # count saturated heteroatoms
    for hetat in mol.GetAtoms():
        idx = hetat.GetIdx()
        if idx in counted_indices: continue
        coo =  hetat.GetTotalDegree()
        symbol = hetat.GetSymbol()
        if symbol == "C": continue
        if symbol == "P" and coo > 3: continue
        name = "%s(%u)" %(symbol, get_nH(hetat))
        if name in ("N(0)", "F(0)"): continue
        counted_indices.add(idx)
        by_type[name].append(idx)
    return dict((group_name, len(by_type[group_name])) for group_name in by_type)


def get_polar_groups(mol):
    global POLARGROUPS
    by_type = defaultdict(list)
    counted_indices = set()
    for group_name in POLARGROUPS:
        group, positions = POLARGROUPS[group_name]
        pattern = Chem.MolFromSmarts(group)
        tuples = mol.GetSubstructMatches(pattern)
        for tup in tuples:
            pos = positions[0]
            atomindex = tup[pos]
            ##>> if set(tup) & counted_indices: continue
            counted_indices |= set(tup)
            by_type[group_name].append(atomindex)
    for bond in mol.GetBonds():
        order = GetBChar(bond)
        if order in ("#", "=", "~"):
            abeg, aend = bond.GetBeginAtom(), bond.GetEndAtom()
            symbols = sorted([abeg.GetSymbol(), aend.GetSymbol()])
            if symbols[0] == symbols[1]: continue  # ADDED to remove C=C and C~C
            tup = (abeg.GetIdx(), aend.GetIdx())
            if set(tup) & counted_indices: continue
            bondsymbol = order.join(symbols)
            counted_indices |= set(tup)
            by_type[bondsymbol].append(order)
    for hetat in mol.GetAtoms():
        idx = hetat.GetIdx()
        ##>> if idx in counted_indices: continue
        coo =  hetat.GetTotalDegree()
        symbol = hetat.GetSymbol()
        #if symbol == "C":
            #voisins = hetat.GetNeighbors()
            #Fs = [v for v in voisins if v.GetSymbol()=="F"]
            #if len(Fs) == 3:
                #by_type["CF3"].append(idx)
        if symbol == "C": continue
        if symbol == "P" and coo > 3: continue
        name = "%s(%u)" %(symbol, get_nH(hetat))
        if name in ("N(0)", "F(0)"): continue
        counted_indices.add(idx)
        by_type[name].append(idx)
 #   print("Grupos para dp:", Counter(dict((group_name, len(by_type[group_name])) for group_name in by_type)))
    return dict((group_name, len(by_type[group_name])) for group_name in by_type)

def get_dic_h(mol):
    dic = defaultdict(int)
    for at in mol.GetAtoms():
        symbol = at.GetSymbol()
        coo = at.GetTotalDegree()
        voisins = [v for v in at.GetNeighbors()]
        if symbol in ("N", "O"):
            if not isNitroN(at, mol):
                dic[symbol] += 1
        if symbol in ("F", "Cl", "Br", "I"):
            dic["X"] += 1
        nH = get_nH(at)
        if nH == 0: continue
        if symbol == "C":
            dic["HC"] += nH
            continue
        if symbol == "N":
            if isAmideN(at, mol):
                dic["HNamide"] += nH
                continue
            if nH == 2:
                dic['H2N'] += 2
            elif nH == 1:
                dic["HN"] += 1
            continue
        if symbol == "O":
            if inCOOH(at, mol):
                dic["HO_COOH"] += nH
            else:
                dic["HO"] += nH
  #  print("Grupos para dh:", dict(Counter(dic)))
    return dic
  
def get_HSPp(mol):
    d = get_polar_groups(mol)
    if set(d) - set(params_p): return float(0)
    eP = sum(params_p[k]*d[k] for k in d)
    return float(eP)

def get_HSPh(mol):
    d = get_dic_h(mol)
    if set(d) - set(params_h): return float(0)
    eH = sum(params_h[k]*d[k] for k in d)
    return float(eH)

def get_atom_code(atom):
    """ return an atom code consistent with the keys of the 'Vinc' dictionary """ 
    # be careful to take into account nD = number of deuterium atoms !
    nD = len([x for x in atom.GetNeighbors() if x.GetMass()>2 and x.GetSymbol()=='H'])
    if nD > 0:
        print('nD %u %s' %(nD, arg))
    code = atom.GetSymbol() + str(atom.GetTotalDegree()) + str(atom.GetTotalNumHs()+nD)
    code += 'a'*int(atom.GetIsAromatic())

    ##############################################################
    # Atribui o nome construído aqui à etiqueta de nome do átomo:
    atom.SetProp("atomNote", code)
    ##############################################################
    return code

def get_Vm_RD(mol):         
    atoms = Counter(get_atom_code(atom) for atom in mol.GetAtoms())
    
    ##########################################################
    #print("Átomos nomeados para Vm e Rm: ", atoms)
    #etiquetas = dict(atoms)
    #[print(chave) for chave in etiquetas]

    #########################################################
    
    missing_atoms = set(atoms) - set(vi)
    if missing_atoms: return None, None
    rings = get_ring_descriptors(mol)
    
    #########################################################
  #  print("Anéis considerados em Vm: ", rings)
    #########################################################

    Vm = sum([atoms[k]*vi[k] for k in atoms])   # atoms contribution
    Vm += sum([rings[k]*vi[k] for k in rings])  # rings contribution
    RD = sum([atoms[k]*ri[k] for k in atoms])   # molar refraction
    return Vm, RD

# UTILITY FOT OUTPUT
def tostr(energy, volume=None):
    if None in (energy, volume): return float(0)
#    if volume == 0: return " %5.1f " %energy
#    return " %5.1f " %sqrt(max(energy/volume,0))
    if volume == 0: return float(energy)
    return float(sqrt(max(energy/volume,0)))

#############################################################################################
# Chamada da função principal para o cálculo do HSP:
def calcula_Mathieu(mol):
    mol = mol    
    # compute molar volume and refractivity
    Vm, RD = get_Vm_RD(mol)
    # compute D-component of HSP
    if Vm is None or RD is None:
        hspD = None
    else:
        hspD = sqrt(93.8 + (2016.0 + 75044.0/Vm) * (RD/Vm)**2.0)  # Equação que determina o parâmetro hspD. 
                                                               # A ref das constantes é a RNN do Mathieu 2018. O mesmo para os demais parâmetros...
    # compute P-component of HSP
    eP = get_HSPp(mol)
    eH = get_HSPh(mol)
    # convert energies to HSP
    hspD, hspP, hspH =  tostr(hspD,0), tostr(eP,Vm), tostr(eH,Vm)
    Vm_Mathieu_sHs, Mathieu_RD, dd_Mathieu, dp_Mathieu, dh_Mathieu = Vm, RD, hspD, hspP, hspH
    return Vm_Mathieu_sHs, Mathieu_RD, dd_Mathieu, dp_Mathieu, dh_Mathieu
###############################################################################################


##################################################################
# Gráfico de espalhamento 3D com esfera de Hansen.
# Função construtora da "esfera de solubilidade"
# Ro é o raio da esfera
# A esfera é posicionada adicionando valores a x, y e z.
# 
m, n = 1, 1 # formato da esfera
pi = np.pi
#Ro = 5 # Raio da esfera
def vec_x(theta, phi, Ro):
     p = Ro + 0.2 * np.sin(m*theta)*np.sin(n*phi)
     # convert from spherical to cartesian
     return p * np.array([
         np.sin(phi) * np.cos(theta),
         np.sin(phi) * np.sin(theta),
         np.cos(phi)
     ])
# each x,y,z has shape (N,N) since it has a value for each (u,v)
u, v = np.mgrid[0:2*pi:200j, 0:pi:100j]
#u, v = np.ogrid[0:2*pi:100j, 0:2*pi:100j]
#x,y,z = vec_x(u, v)
####################################################################

# Funções envolvidas na determinação do HSP por otimização de ensaio experimental:

# Calculadora de erros 
def erros(lista, semaforo, min_solub):
  lista=lista
  semaforo=semaforo
  min_solub = min_solub
  out_ruimDentro = np.where((lista.ensaio >= min_solub) & (lista.RED <= 1), 1, 0)
  out_bomFora = np.where(((lista.ensaio <= min_solub) & (lista.RED > 1)), 1, 0)
  out_total = sum(out_bomFora + out_ruimDentro)
  return out_ruimDentro, out_bomFora, out_total

# Função objetivo da otimização:
def objective(x0, lista, alvo, min_solub):
        dd, dp, dh, Ro = x0
        #print('x0 entrou na função como dd ', dd,', dp ',dp,', dh ',dh,', Ro ',Ro)
        lista = lista
        lab = lista['ensaio']
        alvo = alvo
        min_solub = min_solub
        #print('lab ')
        
        lista['Ra'] = ((4.0*((lista.dd - dd)**2.0)) + ((lista.dp - dp)**2.0) + ((lista.dh - dh)**2.0))**(1/2)
        lista['RED'] = lista['Ra']/Ro
        lista['erros'] = np.where(((lista['RED'] <= 1) & (lista['ensaio'] <= min_solub)) | ((lista['RED'] > 1) & (lista['ensaio'] >= min_solub)), 0, 1 )
        
        Ro = lista[(lista['RED']<1) & (lista['erros']<1)]['Ra'].max()
        
        erro = erros(lista, alvo, min_solub)
        #print('Erro tipo 1, out_ruimDentro: ', erro[0].sum())
        #print('Erro tipo 2, out_bomFora: ', erro[1].sum())
        #print('Erro soma, out_total: ', erro[2])

        lista['Ai_pesos'] = np.where(((lista['RED'] <= 1) & (lista['ensaio'] < min_solub)) | ((lista['RED'] > 1) & (lista['ensaio'] >= min_solub)),
                                     1, (e**-(abs(lista['Ra']-Ro))**(1/lista['ensaio'])) ** (0.58*(e**-(abs(lista['erros']**Ro))))
                                     )
        fit_potencia_erros = np.prod((lista['Ai_pesos']))**(1/lista['ensaio'].sum())

        # Gráfico dataFIT
        fig_obj = px.scatter_3d(lista, x='dd', y='dp', z='dh', opacity=0.7, color=lista['ensaio'].astype('category'),
                                color_discrete_sequence=lista['ensaio'].astype('category'), color_discrete_map={1:'lightgreen', 2:'green', 3:'paleturquoise',4:'yellow', 5:'orange', 6:'red'})
        fig_obj.add_scatter3d(x=[dd], y=[dp], z=[dh], opacity=1)
        #ig_obj.add_mesh3d() = px.line_3d(x=dd, y=dp, z=dh,)
        x,y,z = vec_x(u, v, Ro)
        fig_obj.add_surface(x=x + dd, y=y + dp, z=z + dh, name=alvo, opacity=0.1, cauto=True, showscale = False, hidesurface=False )
       
       # fig_obj.update_layout(scene_aspectmode='cube')
        #fig_obj.show()
        #fig_obj=''

        dataFIT = -fit_potencia_erros
        
        return dataFIT, lista[['ensaio', 'Ai_pesos','Ra', 'RED', 'erros']], fig_obj


In [159]:
##
## Chamadas das funções
# Exemplos para Nevirapina
#

In [160]:
# Para o cálculo usando o esquema de contribuições de grupos de Mathieu:
molsrc='CC1=C2C(=NC=C1)N(C3=C(C=CC=N3)C(=O)N2)C4CC4' # Smile da molécula. Único dado a preencher.

##################################################################################################################################
if 'c' in molsrc or 'C' in molsrc:
  Vm_Mathieu, Rm_Mathieu, dd_Mathieu, dp_Mathieu, dh_Mathieu = calcula_Mathieu(Chem.MolFromSmiles(molsrc))
  Ro_Mathieu = 1600/Vm_Mathieu
  print ('Molécula: ', molsrc)
  print ('Vm: ', Vm_Mathieu, 'Rm: ', Rm_Mathieu)
  print ('dd: ', dd_Mathieu, 'dp: ', dp_Mathieu, 'dh: ', dh_Mathieu, 'Ro: ', Ro_Mathieu)
else:
  print ('A molécula não é a base de carbono, por isso não é possível calcular o HSP utilizando este método')

Molécula:  CC1=C2C(=NC=C1)N(C3=C(C=CC=N3)C(=O)N2)C4CC4
Vm:  199.78000000000003 Rm:  74.57000000000001
dd:  20.664235414015174 dp:  0.0 dh:  10.096763793563426 Ro:  8.008809690659724


In [161]:
solventes.loc[962,:].first

<bound method NDFrame.first of     Smile                           Nome        CAS  Apelido    dd    dp  \
CID                                                                        
962     O                          Water  7732-18-5      NaN  15.5  16.0   
962     O  Water 1% Soluble In - Ro=18.1  7732-18-5      NaN  15.1  20.4   
962     O  Water Complete Misc. (R=13.0)  7732-18-5      NaN  18.1  17.1   

       dh classe  Metodo  
CID                       
962  42.3   solv  Abbott  
962  16.5   solv  Abbott  
962  16.9   solv  Abbott  >

In [163]:
# Para a otimização. Preencher apenas as variáveis alvo, min_solub e dic_exp #######################################################

alvo = 'Nevirapina' # Nome do soluto

# Valor numérico que indica a menor solubilidade aceitável. 0 é totalmente solúvel. 6 é totalmente insolúvel
min_solub = 3 # Valor numérico que indica a menor solubilidade aceitável. 0 é totalmente solúvel. 6 é totalmente insolúvel

# Dicionário com a lista de solventes e seu respectivo efeito sobre o alvo.
# O formato do dado é o número do CID do solvente separado por dois pontos do valor
# do dado do ensaio de solubilidade: 0 é totalmente solúvel. 6 é totalmente insolúvel. 
#(obtido no Handbook do Hansen, pág. 507, Apendix A, Table A3)
# 1. Soluble
# 2. Almost soluble
# 3. Strongly swollen, slight solubility
# 4. Swollen
# 5. Little swelling
# 6. No visible effect
# Pode ser traduzido como
#
# 1. Solúvel
# 2. Quase solúvel
# 3. Fortemente inchado, leve solubilidade
# 4. Inchado
# 5. Pequeno inchaço
# 6. Nenhum efeito visível

dic_exp={ #962:6,
          1031:2,
          180:1,
          6342:2,
          7410:6,
          6569:1,
          263:2,
          241:2,
          7964:2,
          6212:1,
          8078:6,
          6344:2,
          3283:6,
          31374:1,
          6228:1,
          679:1,
          702:1,
          8857:2,
          8900:2,
          8058:6,
          3776:2,
          #887:2,
          6375:1,
          8003:6,
          8028:1,
          1140:2,
          957:1,
          6584:2,
          8103:2,
          6276:2,
          9253:6,
          8452:1,
          11:1,
          7914:6,
          31254:6,
          10907:2,
          10797:2,
          13387:1,
          356:6,
          8894:2,
          8129:1}
print('Total de solventes utilizados no ensaio: ', len(dic_exp.keys()))

######################################################################################################################################
# biblio_solventesAbbott é o dataframe com a lista completa de solventes
# solventes é uma cópia do biblio_solventesAbbott indexado pelo CID
solventes = biblio_solventesAbbott.set_index('CID')

# Um subset deste dataframe, com os parâmetros de Hansen dos solventes e o semáforo
# do ensaio experimental deve ser extraído como subdataframe semáforo para ser enviado á função de otimização
# Para cada chave no dicionário de experimentos dic_exp, cria uma linha no dataframe utilizado na otimização:
df_set = []
for k in dic_exp.keys():
  #print('CID: ', k, 's: ',dic_exp[k],' ', solventes.loc[k,:][['Nome', 'dd', 'dp', 'dh']] ) # Por algum motivo foi preciso tirar uma unidade da chave para bater os resultados!
  # Inclui na lista de listas os dados que se tornarão o novo dataset do ensaio experimental:
  # df_semaforo [CID, ensaio, nome, dd, dp, dh]
  df_set.append([k, dic_exp[k], solventes.loc[k]['Nome'], solventes.loc[k]['dd'], solventes.loc[k]['dp'], solventes.loc[k]['dh']])

# Novo Dataframe com o subset semáforo do ensaio
df_semaforo = pd.DataFrame(df_set, columns = ['CID', 'ensaio', 'nome', 'dd', 'dp', 'dh'])
df_semaforo.fillna(value = 0, inplace = True)


# Condição inicial arbitrária, em geral uma média dos parâmetros dos solventes. Este valor é crítico na otimização
# x0 = [dd, dp, dh, Ro]
# aps é a coluna com o dado do ensaio de solubilidade: 0 é totalmente solúvel. 6 é totalmente insolúvel. 
max_dd = df_semaforo[(df_semaforo['ensaio']<min_solub)].dd.max()
min_dd = df_semaforo[(df_semaforo['ensaio']<min_solub)].dd.min()
delta_dd = max_dd-min_dd

max_dp = df_semaforo[(df_semaforo['ensaio']<min_solub)].dp.max()
min_dp = df_semaforo[(df_semaforo['ensaio']<min_solub)].dp.min()
delta_dp = max_dp-min_dp

max_dh = df_semaforo[(df_semaforo['ensaio']<min_solub)].dh.max()
min_dh = df_semaforo[(df_semaforo['ensaio']<min_solub)].dh.min()
delta_dh = max_dh-min_dh

dd_partida = df_semaforo[(df_semaforo['ensaio']<min_solub)].dd.mean()
dp_partida = df_semaforo[(df_semaforo['ensaio']<min_solub)].dp.mean()
dh_partida = df_semaforo[(df_semaforo['ensaio']<min_solub)].dh.mean()
Ro_partida = sqrt(4*(df_semaforo.dd.mean())**2+(df_semaforo.dp.mean())**2+(df_semaforo.dh.mean())**2)/8
    
x0 = [dd_partida, dp_partida, dh_partida, Ro_partida]

# Chamada da função objetivo de otimização
# Os parâmetros da função são:
# (condições iniciais (x0), a base de dados de solventes com o semáforo, o soluto alvo
# e a referência de mínima solubilidade utilizada no semáforo. No esquema padrão do Hansen, que o semáforo varia de 0 a 6, o mínimo é 2.)
# A função objetivo retorna o fit, HSP+Ro e um gráfico
indice=0
solution=[]
otimiza = minimize(objective, x0, args=(df_semaforo, alvo, min_solub), method='COBYLA', options={'maxiter':1600})
solution.append(otimiza['x']) # Solução de cada otimização
# Imprime um relatório sumarizado das otimizações:
print('Status da otimização %s: %s' % (m, otimiza['message']))
print('Total Evaluations: %d' % otimiza['nfev'])
# evaluate solution
#evaluation = abs(objective(solution[indice], df_semaforo, alvo, min_solub))
evaluation = 999
print('Solution %s: %s(dd, dp, dh, Ro):([%.2f  %.2f  %.2f  %.2f ])' % (m, alvo, solution[indice][0], solution[indice][1], solution[indice][2], solution[indice][3]))
          
#Obs_otimiza = 'Iterações: ' + str(otimiza['nfev']) + ', Resultado da função de otimização (Evaluation)' + str(evaluation)
Ro_otimiza = solution[indice][3]
Ro = Ro_otimiza
print('Ro otimizada: %f' %Ro_otimiza)

x0 = [solution[indice][0], solution[indice][1], solution[indice][2], solution[indice][3]]
fit, data, fig =  objective(x0, df_semaforo, alvo, min_solub)

# Função que calcula o erro
out = erros(lista=data, semaforo=alvo, min_solub=min_solub)

# Relatório dos dados
print('Fit: ', abs(fit))
print('Erros (outliers) total: ', out[2])
print('Erro tipo 1, out_ruimDentro: ', out[0].sum())
print('Erro tipo 2, out_bomFora: ', out[1].sum())
print('Erro soma, out_total: ', out[2])
print('x0: ',x0)
fig.show()
print('')

Total de solventes utilizados no ensaio:  39
Status da otimização 1: Optimization terminated successfully.
Total Evaluations: 56
Solution 1: Nevirapina(dd, dp, dh, Ro):([15.78  8.21  8.46  6.49 ])
Ro otimizada: 6.494609
Fit:  0.9379694259968172
Erros (outliers) total:  14
Erro tipo 1, out_ruimDentro:  0
Erro tipo 2, out_bomFora:  14
Erro soma, out_total:  14
x0:  [15.783840356305554, 8.207374590995785, 8.460224013600799, 6.494609288008411]
